In [ ]:
%cd /app

In [ ]:
import argparse
import os
import sys

os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"

import torch
torch.multiprocessing.set_start_method('spawn')

import jax
from lob.encoding import Vocab, Message_Tokenizer

from lob import inference_no_errcorr as inference
from lob.init_train import init_train_state, load_checkpoint, load_metadata, load_args_from_checkpoint

from lob import inference_no_errcorr as inference
import lob.encoding as encoding
import preproc as preproc

import jax.numpy as jnp
import numpy as onp

from pathlib import Path
import os

import pandas as pd

In [ ]:
import os
from pathlib import Path
from datetime import datetime

def create_next_experiment_folder(save_folder: str) -> Path:
    base = Path(save_folder)
    if not base.exists():
        raise FileNotFoundError(f"Каталога {save_folder!r} не существует")

    max_idx = 0
    for entry in base.iterdir():
        if entry.is_dir() and entry.name.startswith("exp_"):
            parts = entry.name.split("_")
            if len(parts) >= 2 and parts[1].isdigit():
                idx = int(parts[1])
                max_idx = max(max_idx, idx)

    next_idx = max_idx + 1
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    new_folder_name = f"exp_{next_idx}_{timestamp}"
    new_folder = base / new_folder_name
    new_folder.mkdir()
    return new_folder

In [ ]:
# == Parameters for Config == #

save_folder = 'data_saved/' # - need to scan folders in this folder and create next folder 2,3,...,99

batch_size = 4
n_samples = 20

n_gen_msgs = 50 # how many messages to generate into the future
midprice_step_size = 1

num_insertions = 20
num_coolings = 40

EVENT_TYPE_i = 4
DIRECTION_i = 0
order_volume = 75

# ======== #

# scale down to single GPU, single sample inference
bsz = 2 #1, 10
num_devices = 3

n_messages = 500  # length of input sequence to model
book_dim = 501 #b_enc.shape[1] 500+1=501
n_vol_series = 500
sample_top_n = -1

model_size = 'large'
data_dir ='data/test_set/'
sample_all = False # action='store_true'
stock = 'GOOG'  # 'GOOG', 'INTC'

tick_size = 100
sample_all = False
rng_seed = 42


# ======== #

v = Vocab()
n_classes = len(v)
seq_len = n_messages * Message_Tokenizer.MSG_LEN
book_seq_len = n_messages

n_eval_messages = n_gen_msgs
eval_seq_len = n_eval_messages * Message_Tokenizer.MSG_LEN

rng = jax.random.key(rng_seed)
rng, rng_ = jax.random.split(rng)

In [ ]:
save_folder = create_next_experiment_folder(save_folder)
print("Created new experiment directory:", save_folder)

In [ ]:
if stock == 'GOOG':
    # ckpt_path = './checkpoints/treasured-leaf-149_84yhvzjt/' # 0.5 y GOOG, (full model)
    ckpt_path = './checkpoints/denim-elevator-754_czg1ss71/' # large model
    # ckpt_path = './checkpoints/stilted-deluge-759_8g3vqor4'  # small model
elif stock == 'INTC':
    # ckpt_path = './checkpoints/pleasant-cherry-152_i6h5n74c/' # 0.5 y INTC, (full model)
    ckpt_path = './checkpoints/eager-sea-755_2rw1ofs3/'  # large model
else:
    raise ValueError(f'stock {stock} not recognized')

print('Loading metadata:', ckpt_path)
args_ckpt = load_metadata(ckpt_path)

In [ ]:
# load train state from disk

print('Initializing model...')
new_train_state, model_cls = init_train_state(
    args_ckpt,
    n_classes=n_classes,
    seq_len=seq_len,
    book_dim=book_dim,
    book_seq_len=book_seq_len,
)

print('\nLoading model checkpoint...')
ckpt = load_checkpoint(
    new_train_state,
    ckpt_path,
    train=False,
)
state = ckpt['model']

model = model_cls(training=False, step_rescale=1.0)

param_count = sum(x.size for x in jax.tree_leaves(state.params))
print('param count:', param_count)

In [ ]:
# scale down to single GPU, single sample inference
args_ckpt.bsz = bsz #1, 10
args_ckpt.num_devices = num_devices

batchnorm = args_ckpt.batchnorm

data_dir = data_dir + stock
print(f"Directory Path: {data_dir}")

In [ ]:
data_dir = Path(data_dir)
Path(data_dir).mkdir(parents=True, exist_ok=True)
folder_path = Path(data_dir)
file_count = len([f for f in folder_path.iterdir() if f.is_file()])
print(f"There are {file_count} files in the folder {str(data_dir)}.")

In [ ]:
print(n_messages, n_eval_messages)

In [ ]:
from pathlib import Path
import os

# Print current working directory to help verify the path
print(f"Current working directory: {os.getcwd()}")

# Use relative path to data/test_set/GOOG
data_dir = Path("data/test_set/GOOG")

try:
    Path(data_dir).mkdir(parents=True, exist_ok=True)
    print(f"Successfully created or verified directory: {data_dir}")
    
    file_count = len([f for f in Path(data_dir).iterdir() if f.is_file()])
    print(f"There are {file_count} files in the folder {data_dir}.")
except Exception as e:
    print(f"Error: {str(e)}")

ds = inference.get_dataset(data_dir, n_messages, n_eval_messages)

# check 1 by 1

In [ ]:
print(save_folder)

In [ ]:
inference.run_generation_scenario(
    n_samples, 
    batch_size,
    ds,
    rng,
    seq_len,
    n_messages,
    n_gen_msgs,
    state,
    model,
    batchnorm,
    v.ENCODING,
    stock,
    n_vol_series,
    save_folder,
    tick_size,
    sample_top_n,
    sample_all,
    num_insertions,
    num_coolings,
    midprice_step_size,
    EVENT_TYPE_i, 
    DIRECTION_i, 
    order_volume
    )